In [1]:
%reload_ext autoreload
%autoreload 2

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from collections import OrderedDict
import numpy as np
import re
import h5py
from functools import partial
from itertools import product

from calibration import Display


# Wokflow

* General set up (Tab 0):
   
   - Module Number: Specify channel/module numbers that you want to analyse. Must be comma separated.
   - Pulse indices: Pulses that have x-rays. You can provide a range (start:stop:step) or comma separated (1,3,4 etc) or combination of range and individual pulses (1:10:2, 11, 14, 16 etc)
   - Region of interest (ROI x) in x for each module: Only range based (start:stop)
   - Region of interest (ROI y) in y for each module: Only range based (start:stop)
   
* Dark Run (Tab 1):
    
    - Dark Run Folder: (str) /gpfs/exfel/exp/MID/201931/proposal/raw/run_number
    - Train ids: Train indices to take average over. Range based (start:stop). Deafult is all train (:)
    - Process Darak button: To evalute average. This is done in parallel over modules. Doesn't block
      further analysis. Once results (average image and histograms) are available it be displayed automatically.
    - Using Pulses (slider) and module numbers (dropbox) on top one can visulaize data for each pulses or modules.

* Data Visualization (Tab 2):
    
    - Run Folder: (str) /gpfs/exfel/exp/MID/201931/proposal/raw/run_number
    - Train ids: Train indices. Range based (start:stop). Use cautiosly. It loads all train data in memory.
    Also processing is parallized over modules, there p
    
    - Subtract Dark. Once dark average data is available you will be able to check on subtract dark.
    - Load Run: Once things are set up, load the run.
    
    - Fitting Procedure:
    
        - First chose some reasonable peak threshold to filter out number of peaks
        - Chose peak distance to remove very close peaks,
        - Once peaks are chosen, one can click on button Fit Histogram to fit Gaussians. Number of peaks define the number of gaussian functions that will be used to fit the histogram. Generally fitting with 3-4 peaks give reasonable fitting 

In [2]:
config = dict(
    dark_run_folder='/gpfs/exfel/exp/MID/201931/p900091/raw/r0504',
    run_folder='/gpfs/exfel/exp/MID/201931/p900091/raw/r0491',
)

In [3]:
d = Display(config=config)
d.control_panel()

Output(layout=Layout(border='1px solid', height='160px', width='100%'))